## Încapsulări

În exemplul anterior, am lucrat cu vectori de tip bag-of-words de dimensiuni mari, având lungimea `vocab_size`, și am convertit explicit din vectori de reprezentare pozițională de dimensiuni mici în reprezentări sparse de tip one-hot. Această reprezentare one-hot nu este eficientă din punct de vedere al memoriei și, în plus, fiecare cuvânt este tratat independent de celelalte, adică vectorii codificați one-hot nu exprimă nicio similaritate semantică între cuvinte.

În această unitate, vom continua să explorăm setul de date **News AG**. Pentru început, să încărcăm datele și să obținem câteva definiții din notebook-ul anterior.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Ce este embedding-ul?

Ideea de **embedding** este de a reprezenta cuvintele prin vectori densi de dimensiuni mai mici, care reflectă într-un fel semnificația semantică a unui cuvânt. Vom discuta mai târziu cum să construim embedding-uri semnificative pentru cuvinte, dar deocamdată să ne gândim la embedding-uri ca la o modalitate de a reduce dimensiunea unui vector de cuvânt.

Astfel, un strat de embedding ar lua un cuvânt ca intrare și ar produce un vector de ieșire cu dimensiunea specificată `embedding_size`. Într-un fel, este foarte similar cu stratul `Linear`, dar în loc să ia un vector codificat one-hot, va putea să primească un număr de cuvânt ca intrare.

Folosind stratul de embedding ca prim strat în rețeaua noastră, putem trece de la modelul bag-of-words la modelul **embedding bag**, unde mai întâi convertim fiecare cuvânt din textul nostru în embedding-ul corespunzător, iar apoi calculăm o funcție de agregare peste toate aceste embedding-uri, cum ar fi `sum`, `average` sau `max`.

![Imagine care arată un clasificator embedding pentru cinci cuvinte dintr-o secvență.](../../../../../translated_images/ro/embedding-classifier-example.b77f021a7ee67eee.webp)

Rețeaua noastră neuronală de clasificare va începe cu un strat de embedding, urmat de un strat de agregare și un clasificator liniar deasupra acestuia:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Gestionarea dimensiunii variabile a secvenței

Ca rezultat al acestei arhitecturi, minibatch-urile pentru rețea ar trebui să fie create într-un mod specific. În unitatea anterioară, când foloseam metoda bag-of-words, toate tensori BoW dintr-un minibatch aveau aceeași dimensiune `vocab_size`, indiferent de lungimea reală a secvenței text. Odată ce trecem la încorporări de cuvinte, ajungem să avem un număr variabil de cuvinte în fiecare eșantion de text, iar atunci când combinăm aceste eșantioane în minibatch-uri, va trebui să aplicăm un padding.

Acest lucru poate fi realizat folosind aceeași tehnică de furnizare a funcției `collate_fn` sursei de date:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Antrenarea clasificatorului de embedding

Acum că am definit un dataloader corespunzător, putem antrena modelul folosind funcția de antrenare pe care am definit-o în unitatea precedentă:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Notă**: Aici antrenăm doar pentru 25k de înregistrări (mai puțin de un întreg epoc) pentru a economisi timp, dar puteți continua antrenamentul, scrie o funcție pentru a antrena pentru mai multe epoci și experimentați cu parametrul ratei de învățare pentru a obține o acuratețe mai mare. Ar trebui să puteți ajunge la o acuratețe de aproximativ 90%.


### Stratul EmbeddingBag și Reprezentarea Secvențelor de Lungime Variabilă

În arhitectura anterioară, a fost necesar să completăm toate secvențele la aceeași lungime pentru a le încadra într-un minibatch. Aceasta nu este cea mai eficientă metodă de a reprezenta secvențele de lungime variabilă - o altă abordare ar fi utilizarea unui vector de **offset**, care ar conține offset-urile tuturor secvențelor stocate într-un singur vector mare.

![Imagine care arată o reprezentare a secvențelor cu offset](../../../../../translated_images/ro/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: În imaginea de mai sus, este prezentată o secvență de caractere, dar în exemplul nostru lucrăm cu secvențe de cuvinte. Totuși, principiul general de reprezentare a secvențelor cu un vector de offset rămâne același.

Pentru a lucra cu reprezentarea prin offset, utilizăm stratul [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Este similar cu `Embedding`, dar ia ca intrare un vector de conținut și un vector de offset și include, de asemenea, un strat de agregare, care poate fi `mean`, `sum` sau `max`.

Iată o rețea modificată care utilizează `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Pentru a pregăti setul de date pentru antrenare, trebuie să furnizăm o funcție de conversie care va pregăti vectorul de offset:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Notă, că spre deosebire de toate exemplele anterioare, rețeaua noastră acceptă acum doi parametri: vectorul de date și vectorul de offset, care au dimensiuni diferite. În mod similar, loader-ul nostru de date ne oferă acum 3 valori în loc de 2: atât vectorii de text, cât și cei de offset sunt furnizați ca caracteristici. Prin urmare, trebuie să ajustăm ușor funcția noastră de antrenament pentru a ține cont de acest lucru:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Încapsulări Semantice: Word2Vec

În exemplul nostru anterior, stratul de învățare al modelului a fost antrenat să mapeze cuvintele la o reprezentare vectorială, însă această reprezentare nu avea prea multă semnificație semantică. Ar fi util să învățăm o astfel de reprezentare vectorială, în care cuvintele similare sau sinonime să corespundă unor vectori care sunt apropiați unul de celălalt în funcție de o anumită distanță vectorială (de exemplu, distanța euclidiană).

Pentru a realiza acest lucru, trebuie să pre-antrenăm modelul de învățare pe o colecție mare de texte într-un mod specific. Una dintre primele metode de antrenare a încapsulărilor semantice se numește [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Aceasta se bazează pe două arhitecturi principale utilizate pentru a produce o reprezentare distribuită a cuvintelor:

 - **Continuous bag-of-words** (CBoW) — în această arhitectură, modelul este antrenat să prezică un cuvânt pe baza contextului din jur. Având ngrama $(W_{-2},W_{-1},W_0,W_1,W_2)$, scopul modelului este să prezică $W_0$ pe baza $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** este opusul CBoW. Modelul folosește fereastra de cuvinte contextuale din jur pentru a prezice cuvântul curent.

CBoW este mai rapid, în timp ce skip-gram este mai lent, dar oferă o reprezentare mai bună pentru cuvintele rare.

![Imagine care arată algoritmii CBoW și Skip-Gram pentru conversia cuvintelor în vectori.](../../../../../translated_images/ro/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Pentru a experimenta cu încapsulările word2vec pre-antrenate pe setul de date Google News, putem folosi biblioteca **gensim**. Mai jos găsim cuvintele cele mai similare cu 'neural'.

> **Note:** Când creați pentru prima dată vectori de cuvinte, descărcarea acestora poate dura ceva timp!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Putem, de asemenea, să calculăm încorporările vectoriale din cuvânt, pentru a fi utilizate în antrenarea modelului de clasificare (afișăm doar primele 20 de componente ale vectorului pentru claritate):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Lucrul grozav despre încorporările semantice este că poți manipula codificarea vectorială pentru a schimba semantica. De exemplu, putem cere să găsim un cuvânt, a cărui reprezentare vectorială să fie cât mai apropiată de cuvintele *rege* și *femeie*, și cât mai îndepărtată de cuvântul *bărbat*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Atât CBoW, cât și Skip-Grams sunt încorporări „predictive”, în sensul că iau în considerare doar contexte locale. Word2Vec nu profită de contextul global.

**FastText** se bazează pe Word2Vec prin învățarea reprezentărilor vectoriale pentru fiecare cuvânt și pentru n-gramele de caractere găsite în interiorul fiecărui cuvânt. Valorile reprezentărilor sunt apoi mediate într-un singur vector la fiecare pas de antrenament. Deși acest lucru adaugă multă calculare suplimentară în etapa de pre-antrenare, permite încorporărilor de cuvinte să encodeze informații la nivel de sub-cuvânt.

O altă metodă, **GloVe**, valorifică ideea matricei de co-apariție, utilizând metode neuronale pentru a descompune matricea de co-apariție în vectori de cuvinte mai expresivi și non-liniari.

Poți experimenta cu exemplul schimbând încorporările în FastText și GloVe, deoarece gensim suportă mai multe modele diferite de încorporare a cuvintelor.


## Utilizarea Embedding-urilor Pre-Antrenate în PyTorch

Putem modifica exemplul de mai sus pentru a pre-popula matricea din stratul nostru de embedding cu embedding-uri semantice, cum ar fi Word2Vec. Trebuie să ținem cont de faptul că vocabularele embedding-urilor pre-antrenate și ale corpusului nostru de text probabil nu se vor potrivi, așa că vom inițializa greutățile pentru cuvintele lipsă cu valori random:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Acum să antrenăm modelul nostru. Rețineți că timpul necesar pentru a antrena modelul este semnificativ mai mare decât în exemplul anterior, datorită dimensiunii mai mari a stratului de încorporare și, astfel, unui număr mult mai mare de parametri. De asemenea, din acest motiv, s-ar putea să fie nevoie să antrenăm modelul nostru pe mai multe exemple dacă dorim să evităm supraînvățarea.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

În cazul nostru, nu observăm o creștere semnificativă a acurateței, cel mai probabil din cauza vocabularului destul de diferit.  
Pentru a depăși problema vocabularului diferit, putem folosi una dintre următoarele soluții:  
* Re-antrenarea modelului word2vec pe vocabularul nostru  
* Încărcarea setului nostru de date folosind vocabularul din modelul word2vec pre-antrenat. Vocabularul utilizat pentru încărcarea setului de date poate fi specificat în timpul încărcării.  

A doua abordare pare mai ușoară, mai ales deoarece cadrul `torchtext` din PyTorch conține suport integrat pentru embeddings. Putem, de exemplu, să instanțiem un vocabular bazat pe GloVe în următorul mod:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Vocabularul încărcat are următoarele operațiuni de bază:  
* Dicționarul `vocab.stoi` ne permite să convertim un cuvânt în indexul său din dicționar  
* `vocab.itos` face opusul - convertește un număr în cuvânt  
* `vocab.vectors` este matricea de vectori de încorporare, așa că pentru a obține încorporarea unui cuvânt `s`, trebuie să folosim `vocab.vectors[vocab.stoi[s]]`  

Iată un exemplu de manipulare a încorporărilor pentru a demonstra ecuația **kind-man+woman = queen** (a trebuit să ajustez puțin coeficientul pentru a funcționa):  


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Pentru a antrena clasificatorul folosind acele embeddings, mai întâi trebuie să codificăm setul nostru de date folosind vocabularul GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Așa cum am văzut mai sus, toate încorporările vectoriale sunt stocate în matricea `vocab.vectors`. Acest lucru face extrem de ușoară încărcarea acelor greutăți în greutățile stratului de încorporare folosind copierea simplă:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Unul dintre motivele pentru care nu observăm o creștere semnificativă a acurateței este faptul că unele cuvinte din setul nostru de date lipsesc din vocabularul GloVe pre-antrenat și, prin urmare, sunt practic ignorate. Pentru a depăși acest fapt, putem antrena propriile noastre embedding-uri pe setul nostru de date.


## Încadrări Contextuale

O limitare importantă a reprezentărilor tradiționale de încadrare pre-antrenate, cum ar fi Word2Vec, este problema dezambiguizării sensului cuvintelor. Deși încadrările pre-antrenate pot surprinde o parte din sensul cuvintelor în context, fiecare posibil sens al unui cuvânt este codificat în aceeași încadrare. Acest lucru poate cauza probleme în modelele ulterioare, deoarece multe cuvinte, cum ar fi cuvântul „play”, au sensuri diferite în funcție de contextul în care sunt utilizate.

De exemplu, cuvântul „play” în aceste două propoziții are sensuri destul de diferite:
- Am fost la o **piesă** de teatru.
- John vrea să se **joace** cu prietenii săi.

Încadrările pre-antrenate de mai sus reprezintă ambele sensuri ale cuvântului „play” în aceeași încadrare. Pentru a depăși această limitare, trebuie să construim încadrări bazate pe **modelul lingvistic**, care este antrenat pe un corpus mare de text și *știe* cum pot fi utilizate cuvintele în contexte diferite. Discutarea încadrărilor contextuale depășește scopul acestui tutorial, dar vom reveni la ele atunci când vom vorbi despre modelele lingvistice în unitatea următoare.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
